In [1]:
from shared.preface import *
import shared.functions as fct
import shared.my_units as my
import shared.control_center as CC

## Exploring snapshot_0036

In [ ]:
# print(os.getcwd())

f = h5py.File('/home/fabian/ownCloud/sim_data/snapshot_0036.hdf5')

def printname(name):
    print(name)

# f.visit(printname)
print(f.keys())

In [ ]:
dset = f['DMParticles/Coordinates']

# dset.visit(printname)

print(dset.shape)

## Visualize DM mass distribution

In [ ]:
from swiftsimio import load
from swiftsimio.visualisation.projection import project_pixel_grid
from swiftsimio.visualisation.smoothing_length_generation import generate_smoothing_lengths

data = load('/home/fabian/ownCloud/sim_data/snapshot_0036.hdf5')

# Generate smoothing lengths for the dark matter
data.dark_matter.smoothing_length = generate_smoothing_lengths(
    data.dark_matter.coordinates,
    data.metadata.boxsize,
    kernel_gamma=1.8,
    neighbours=57,
    speedup_fac=2,
    dimension=3,
)

# Project the dark matter mass
dm_mass = project_pixel_grid(
    # Note here that we pass in the dark matter dataset not the whole
    # data object, to specify what particle type we wish to visualise
    data=data.dark_matter,
    boxsize=data.metadata.boxsize,
    resolution=1024,
    project="masses",
    parallel=True,
    region=None
)

In [ ]:
from matplotlib.pyplot import imsave
from matplotlib.colors import LogNorm

# Everyone knows that dark matter is purple
imsave("sim_data/dm_mass_map.png", LogNorm()(dm_mass), cmap="inferno")

## Calculate halo mass function (HMF)

In [37]:
from colossus.cosmology import cosmology
cosmology.setCosmology('planck13');
from colossus.lss import mass_function

def HMF(halo):

    dlogm = 0.2
    bins = 10 ** (np.arange(6, 15, dlogm))
    V = 12 ** 3 # Mpc

    # Load the data
    g = h5py.File('/home/fabian/ownCloud/sim_data/subhalo_0036.properties', 'r')
    mass = g["Mass_200crit"][:] * 1e10  # convert to Msun
    print(mass.shape)

    if halo == 'subhalo':
        subtype = g["Structuretype"][:]
        subhalo = subtype > 10
        mass = mass[subhalo]

    binnedmass, massrange = np.histogram(mass, bins=bins)
    print(binnedmass.shape, binnedmass[10:25])

    massnlarger = np.zeros(len(binnedmass))
    for i in range(0, len(massnlarger)):
        massnlarger[i] = np.sum(binnedmass[i:])

    f = h5py.File('/home/fabian/ownCloud/sim_data/snapshot_0036.hdf5', 'r')
    cosmo = f["Cosmology"]
    redshift = cosmo.attrs["Redshift"][0]
    mass = f["PartType1/Masses"][:]
    mass = mass[0] * 1e10 # convert to Msun
    limit = 100 * mass

    # Determine the HMF #? why do these lines of code generate the HMF?
    errormassn = massnlarger ** 0.5
    numbden = massnlarger / V
    numbdenerr = errormassn / V
    massplot = (massrange[:-1] + massrange[1:]) / 2
    dernumbden = -np.diff(numbden) / np.diff(np.log10(massplot))
    dererr = 2 ** 0.5 / dlogm * (numbdenerr[:-1] + numbdenerr[1:]) / 2

    M = (massplot[:-1] + massplot[1:]) / 2.
    dndM = dernumbden
    error = dererr

    return M, dndM, error, limit, redshift


def make_HMF():

    M, dndM, error, limit, redshift = HMF('halo')

    plt.plot(M, dndM, label="snapshot of z=0",lw=2,color='tab:green')
    # plt.fill_between(M, dndM - error, dndM + error, alpha=0.4,color='tab:green')

    h = 0.67
    M = 10 ** np.arange(7, 16, 0.2)

    # Tinker version.
    mfunc = mass_function.massFunction(
        M / h, redshift, mdef='200c', model='tinker08', q_out='dndlnM'
        )

    # Sim version.
    plt.loglog(M, mfunc / h**3, '-', label='Tinker+08', color='black', zorder=1)

    # plt.ylim(1e-7, 1e0)
    # plt.xlim(10 ** 9, 10 ** 15)
    # plt.axvline(x=limit, linestyle="--", lw=1, color="grey")

    plt.xlabel("M${}_{200,\mathrm{crit}}$ ($M_\odot$)")
    plt.ylabel("dn/d($\log$10(M${}_{200}$) (Mpc$^{-3}$)")
    plt.legend(loc='upper right' ,prop={'size': 10})
    plt.grid("True")
    plt.savefig('sim_data/HMF_0036.pdf', bbox_inches='tight')#, dpi=200)
    plt.close()

    # M, dndM, error, limit, redshift = HMF('subhalo')

    # plt.figure()
    # ax = plt.subplot(1,1,1)
    # plt.grid("True")

    # plt.plot(M, dndM, label="SWIFT",lw=2,color='tab:blue')
    # plt.fill_between(M, dndM - error, dndM + error, alpha=0.4,color='tab:blue')

    # plt.xscale("log")
    # # plt.ylim(1e-7, 1e0)
    # # plt.xlim(10 ** 9, 10 ** 15)
    # plt.axvline(x=limit, linestyle="--", lw=1, color="grey")

    # h = 0.67
    # M = np.arange(7, 16, 0.2)
    # M = 10**M
    # mfunc = mass_function.massFunction(M / h, redshift, mdef = '200c', model = 'tinker08', q_out = 'dndlnM')
    # plt.plot(M, mfunc / h**3, '-', label = 'Tinker et al. (2008)',color='black',zorder=1)

    # plt.xlabel("M${}_{200,\mathrm{crit}}$ ($M_\odot$)")
    # plt.ylabel("(Subhalo) dn/d($\log$10(M${}_{200}$) (Mpc$^{-3}$)")
    # ax.tick_params(direction='in',axis='both',which='both',pad=4.5)
    # plt.yscale("log")
    # # plt.legend(loc=[0.45,0.8],labelspacing=0.2,handlelength=1.5,handletextpad=0.4,frameon=False)
    # plt.legend()
    # ax.tick_params(direction='in', axis='both', which='both', pad=4.5)
    # plt.savefig('sim_data/SubHMF_0036.pdf')#, dpi=200)
    # plt.close()

make_HMF()

(6414,)
(44,) [ 104  350 1258 1395  970  705  436  301  197  132   78   44   43   16
   23]
